# Code to make "train.pkl" and "dev.pkl" 

In [2]:
import pandas as pd
import numpy as np
import math
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re,string,unicodedata
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
from random import choice

C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

We use all the questions, its mapping with best reviews and Vocab file to generate these files

In [6]:
questions = pd.read_csv('Questions_126Products.csv')
mapping = pd.read_csv('mapping_formatted.csv')
vocab = pd.read_csv('Vocab_file.csv')

In [7]:
questions.shape, mapping.shape

((737, 3), (735, 6))

In [8]:
questions.head()

,Question_ID,Product ID,Question Text
0,0,1,Will this keep a strong 6 year old out of my c...
1,1,1,What's the difference between this and the reg...
2,2,1,I have these locks on my cabinets and love the...
3,3,1,Safety 1st Magnetic Tot Lok Deluxe Starter Set...
4,4,1,I seem to have misplaced my magnet. Is there ...


In [9]:
mapping.head()

,Ques_ID,Rev_1,Rev_2,Rev_3,Rev_4,Rev_5
0,0,10,23.0,11.0,53.0,27.0
1,1,46,11.0,24.0,10.0,64.0
2,2,57,31.0,33.0,67.0,26.0
3,3,57,67.0,11.0,8.0,48.0
4,4,30,27.0,11.0,64.0,58.0


In [88]:
vocab.head(2)

,Unnamed: 0,0
0,the,1
1,it,2


In [89]:
vocab.columns = ["words", "num"]

In [90]:
vocab.head(2)

,words,num
0,the,1
1,it,2


In [91]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [92]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\miman\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Lemmatization of questions

In [93]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    for i in text.split():
        pos = pos_tag([i.strip()])
        word = lemmatizer.lemmatize(i.strip(),get_simple_pos(pos[0][1]))
        final_text.append(word.lower())
    return " ".join(final_text)

In [94]:
questions['Q'] = questions["Question Text"].apply(lemmatize_words)

In [95]:
questions.shape

(737, 4)

In [96]:
questions.Q = [re.sub(r'\d+', '', x) for x in questions.Q]

### Working with vocab file

In [97]:
vocab=vocab.set_index('words')
vocab.head()

,num
words,
the,1
it,2
and,3
to,4
a,5


In [98]:
vocab = vocab.to_dict()
vocab.keys()

dict_keys(['num'])

In [99]:
len(vocab["num"])

27150

In [100]:
vocab = vocab["num"]

### Tokenization and labelling of questions

In [101]:
tokenizer = Tokenizer()
tokenizer.word_index = vocab
word_index = tokenizer.word_index

In [102]:
ques_temp = list(questions.Q)

In [103]:
len(ques_temp)

737

In [104]:
ques_sequences = tokenizer.texts_to_sequences(ques_temp)
ques_sequences[0]

[61,
 9,
 110,
 5,
 579,
 121,
 54,
 42,
 11,
 12,
 2745,
 31,
 16,
 1642,
 131,
 3,
 281,
 13,
 31,
 35,
 299,
 12,
 1386,
 3,
 1581,
 1213,
 15,
 1,
 313,
 6,
 27,
 164,
 8,
 229,
 13,
 34,
 1072,
 107,
 4,
 50,
 10,
 12,
 2745]

In [105]:
len(ques_sequences)

737

### Adding best reviews to review_text (train.pkl)

In [111]:
questions.tail()

,Question_ID,Product ID,Question Text,Q
732,732,125,Does this work with a Graco Snugride 32 car seat?,does this work with a graco snugride car seat?
733,733,125,does this base fit the Graco Alano Classic Con...,do this base fit the graco alano classic conne...
734,734,125,How difficult is this for a grandmother to ins...,how difficult be this for a grandmother to ins...
735,735,125,Does this base fit a 2011 Graco Snugride 35 ca...,does this base fit a graco snugride carseat?
736,736,125,Does this base fit \'Graco UrbanLite Classic C...,does this base fit \'graco urbanlite classic c...


In [106]:
mapping.head()

,Ques_ID,Rev_1,Rev_2,Rev_3,Rev_4,Rev_5
0,0,10,23.0,11.0,53.0,27.0
1,1,46,11.0,24.0,10.0,64.0
2,2,57,31.0,33.0,67.0,26.0
3,3,57,67.0,11.0,8.0,48.0
4,4,30,27.0,11.0,64.0,58.0


In [107]:
 questions.shape[0], mapping.shape[0]

(737, 735)

This tells us that all the questions are not present in mapping. Reason - we do not have reviews corresponding to these questions

In [108]:
questions.dtypes

Question_ID       int64
Product ID        int64
Question Text    object
Q                object
dtype: object

### Creating train.pkl file

In [112]:
reviews_text = []

for i in range(151, questions.shape[0]):
    if ((i in questions["Question_ID"].values) & (i in mapping["Ques_ID"].values)):
        k = mapping[mapping["Ques_ID"]==i].index
        k=int(k[0])
        print(k)
        dict_1 = {}
        
        temp_list=[]
        for j in range(1,6):
            if (math.isnan(mapping.iloc[k:k+1,j:j+1].values[0][0]) == False) & (mapping.iloc[k:k+1,j:j+1].values[0][0]>0)  :
                temp_list.append(mapping.iloc[k:k+1,j:j+1].values[0][0].astype(int))
                
        dict_1["question"] = ques_sequences[i]
        dict_1["answers"] = temp_list    
        reviews_text.append(dict_1)

151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400


In [113]:
len(reviews_text)

584

In [114]:
reviews_text

[{'question': [35, 6, 9885, 63], 'answers': [8553, 8476, 8660, 8539, 8454]},
 {'question': [123, 27, 1, 637, 1939, 1414, 105, 42, 11],
  'answers': [8543, 8677, 8580, 8536, 8510]},
 {'question': [27, 63, 840, 27], 'answers': [8663, 8553, 8586, 8578, 8646]},
 {'question': [6,
   837,
   737,
   5,
   346,
   329,
   11,
   8027,
   688,
   29,
   1,
   349,
   3,
   1,
   1089,
   212,
   7419,
   25,
   63,
   1224,
   568,
   712],
  'answers': [8472, 8496, 8621, 8514, 8609]},
 {'question': [27,
   63,
   688,
   180,
   8,
   5,
   56,
   54,
   6,
   167,
   2,
   4,
   191,
   51,
   20,
   41,
   338,
   116,
   6,
   167,
   5,
   180,
   1243],
  'answers': [8449, 8656, 8588, 8496, 8561]},
 {'question': [7, 1, 3218, 435, 1651, 15, 1, 581],
  'answers': [8543, 8463, 8605, 8604, 8582]},
 {'question': [35, 6, 39, 63, 8, 12, 178, 464, 24, 58, 169],
  'answers': [8504, 8589, 8449, 8491, 8496]},
 {'question': [133,
   499,
   27,
   1,
   502,
   1517,
   508,
   34,
   5,
   727,
   

In [116]:
#pickle.dump(reviews_text, open('train.pkl', 'wb'))

### Creating dev.pkl file

In [117]:
reviews = pd.read_csv("Reviews_126Products.csv")

In [130]:
reviews.head()

,Review_ID,Product ID,Review_Text
0,0,1,'not really as portrayed on video
1,1,1,'HORRIBLE installation experience. Installed p...
2,2,1,'I wish we'd bought the cheaper \'push latch\'...
3,3,1,"'The concept is great, but these things are a ..."
4,4,1,'My husband installed 4 of these locks on my s...


In [147]:
reviews_text = []

for i in range(0, 151):
    if ((i in questions["Question_ID"].values) & (i in mapping["Ques_ID"].values)):
        k = mapping[mapping["Ques_ID"]==i].index
        k=int(k[0])
        print(k)
       
        dict_1 = {}
       
        temp_list=[]
        for j in range(1,6):
            if ((math.isnan(mapping.iloc[k:k+1,j:j+1].values[0][0]) == False) & (mapping.iloc[k:k+1,j:j+1].values[0][0] > 0)):
                temp_list.append(mapping.iloc[k:k+1,j:j+1].values[0][0].astype(int))
        baddy = []
        
        pdtid = questions["Product ID"][i]
        reviews_pdt = reviews[reviews["Product ID"]==pdtid]
        reviews_list_pdt = reviews_pdt["Review_ID"].unique()
        
        #for z in range (0,500 - len(temp_list)):
        baddy.append([t for t in reviews_list_pdt if t not in temp_list])
        
        dict_1["bad"] = baddy[0]
        dict_1["good"] = temp_list
        dict_1["question"] = ques_sequences[i]
           
        reviews_text.append(dict_1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [148]:
len(reviews_text)

151

In [149]:
reviews_text

[{'bad': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   24,
   25,
   26,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70],
  'good': [10, 23, 11, 53, 27],
  'question': [61,
   9,
   110,
   5,
   579,
   121,
   54,
   42,
   11,
   12,
   2745,
   31,
   16,
   1642,
   131,
   3,
   281,
   13,
   31,
   35,
   299,
   12,
   1386,
   3,
   1581,
   1213,
   15,
   1,
   313,
   6,
   27,
   164,
   8,
   229,
   13,
   34,
   1072,
   107,
   4,
   50,
   10,
   12,
   2745]},
 {'bad': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   25,
   26,
   27,
   28

In [150]:
#pickle.dump(reviews_text, open('dev.pkl', 'wb'))